# **Reglas de asociación - Marcos Damián Pool Canul**

In [39]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from itertools import combinations
import pandas as pd

In [40]:
df = pd.read_csv("../files/Walmart.csv")
print("Datos originales:", df.values.shape)

Datos originales: (541909, 4)


In [41]:
df.head(10)

,Description,Quantity,UnitPrice,CustomerID
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0
1,WHITE METAL LANTERN,6,3.39,17850.0
2,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0
4,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0
5,SET 7 BABUSHKA NESTING BOXES,2,7.65,17850.0
6,GLASS STAR FROSTED T-LIGHT HOLDER,6,4.25,17850.0
7,HAND WARMER UNION JACK,6,1.85,17850.0
8,HAND WARMER RED POLKA DOT,6,1.85,17850.0
9,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,13047.0


## **Limpieza de los datos**

In [42]:
# Se eliminan los registros donde el valor de CustomerID es nulo
df.dropna(subset=['CustomerID'], inplace=True)

In [43]:
# Se adecua los índices para que no tomen en cuenta los registros que se borraron.
df.reset_index(drop=True, inplace=True)

In [44]:
# Se crea una columna en la que aumenta el valor cada vez que cambia el valor del CustomerID
df['Customer'] = (df['CustomerID'] != df['CustomerID'].shift()).cumsum()

In [45]:
# Eliminamos las columnas que no se van a utilizar
df = df.drop(['Quantity', 'UnitPrice', 'CustomerID'], axis=1)

In [46]:
# Cambiar el nombre de la columna "Description" a "Product"
df.rename(columns={'Description': 'Product'}, inplace=True)

In [47]:
# Guardamos el nuevo dataset en un archivo csv
df.to_csv('../files/Walmart_prepared_data.csv', index=False)

In [48]:
# Mostramos los primeros 10 registros
df.head(10)

,Product,Customer
0,WHITE HANGING HEART T-LIGHT HOLDER,1
1,WHITE METAL LANTERN,1
2,CREAM CUPID HEARTS COAT HANGER,1
3,KNITTED UNION FLAG HOT WATER BOTTLE,1
4,RED WOOLLY HOTTIE WHITE HEART.,1
5,SET 7 BABUSHKA NESTING BOXES,1
6,GLASS STAR FROSTED T-LIGHT HOLDER,1
7,HAND WARMER UNION JACK,1
8,HAND WARMER RED POLKA DOT,1
9,ASSORTED COLOUR BIRD ORNAMENT,2


In [49]:
# Leemos el nuevo dataset limpio
df = pd.read_csv("../files/walmart_prepared_data.csv")
print(df.values.shape)

(406829, 2)


## **Articulos**

Se realizan las siguientes operaciones en un conjunto de datos:

1. Se crea un conjunto `I` a partir de los valores únicos en la columna "Product" del dataframe `df`, eliminando duplicados.
2. Se calcula la cantidad de elementos únicos en `I` y la almacena en la variable `cantidad`.
3. Imprime la cantidad de productos únicos encontrados.

In [50]:
I = set(df["Product"])
I = sorted(set(df["Product"]))

cantidad = len(I)

print(cantidad)
I[:10]

3896


[' 4 PURPLE FLOCK DINNER CANDLES',
 " 50'S CHRISTMAS GIFT BAG LARGE",
 ' DOLLY GIRL BEAKER',
 ' I LOVE LONDON MINI BACKPACK',
 ' I LOVE LONDON MINI RUCKSACK',
 ' NINE DRAWER OFFICE TIDY',
 ' OVAL WALL MIRROR DIAMANTE ',
 ' RED SPOT GIFT BAG LARGE',
 ' SET 2 TEA TOWELS I LOVE LONDON ',
 ' SPACEBOY BABY GIFT SET']

## **Transacciones**

Esta seccion de código se ocupa de transformar el conjunto de datos para agrupar los productos comprados por cada cliente, facilitando la identificación de los artículos en cada transacción. Veamos detalladamente lo que hace cada parte:

In [51]:
# Realizamos la transformación del conjunto de datos.
T = df.groupby(['Customer']).agg({'Product': set})

In [52]:
T.head(10)

,Product
Customer,
1,"{WHITE METAL LANTERN, HAND WARMER RED POLKA DO..."
2,"{RECIPE BOX WITH METAL HEART, RED COAT RACK PA..."
3,"{LUNCH BOX I LOVE LONDON, CHARLOTTE BAG DOLLY ..."
4,{PAPER CHAIN KIT 50'S CHRISTMAS }
5,"{WHITE METAL LANTERN, RETRO COFFEE MUGS ASSORT..."
6,{VICTORIAN SEWING BOX LARGE}
7,"{WHITE METAL LANTERN, RETRO COFFEE MUGS ASSORT..."
8,"{HOT WATER BOTTLE TEA AND SYMPATHY, RED HANGIN..."
9,"{HAND WARMER UNION JACK, HAND WARMER RED POLKA..."


In [53]:
# Renombramos la columna "Product" a "Transaction"
T = T.rename(columns={"Product": "Transaction"})
T

,Transaction
Customer,
1,"{WHITE METAL LANTERN, HAND WARMER RED POLKA DO..."
2,"{RECIPE BOX WITH METAL HEART, RED COAT RACK PA..."
3,"{LUNCH BOX I LOVE LONDON, CHARLOTTE BAG DOLLY ..."
4,{PAPER CHAIN KIT 50'S CHRISTMAS }
5,"{WHITE METAL LANTERN, RETRO COFFEE MUGS ASSORT..."
...,...
20769,"{CLASSIC BICYCLE CLIPS , SET OF 6 RIBBONS PERF..."
20770,"{6 CHOCOLATE LOVE HEART T-LIGHTS, RED FLOCK LO..."
20771,"{GREY HEART HOT WATER BOTTLE, ASSORTED BOTTLE ..."


## **Conteo**

Esta sección describe un proceso para calcular cuántas transacciones contienen un conjunto específico de productos. Este proceso es crucial para las reglas de asociación, ya que necesitamos saber con qué frecuencia aparecen conjuntos de ítems juntos en las transacciones. Vamos a desglosarlo paso a paso:

In [54]:
# Este conjunto representa el grupo de ítems cuya frecuencia de aparición en las transacciones queremos determinar.
X = set(["LOVE BUILDING BLOCK WORD"])

In [55]:
# Aquí, aplicamos una función a cada elemento en la columna "Transaction" del dataframe T.
X_subset =  T["Transaction"].apply(lambda t: X.issubset(t))

print(X_subset)

Customer
1        False
2         True
3        False
4        False
5        False
         ...  
20769    False
20770    False
20771    False
20772    False
20773    False
Name: Transaction, Length: 20773, dtype: bool


In [56]:
# Sumar estos valores nos da el número total de transacciones que contienen todos los ítems en X.
sum(X_subset)

557

### **Definimos una función**

Para hacer este proceso más eficiente y reutilizable, se define una función conteo:

- `X`: El conjunto de ítems cuya presencia en las transacciones queremos contar.
- `T`: El dataframe de transacciones, donde cada transacción está en la columna "Transaction".
- La función aplica la misma lógica descrita anteriormente, pero ahora de forma más flexible, permitiendo pasar diferentes conjuntos de `X` para su evaluación en `T`.

### **Uso de la Función**

- `conteo(set(["LOVE BUILDING BLOCK WORD"]), T)`: Aquí se cuenta nuevamente las transacciones que contienen "LOVE BUILDING BLOCK WORD", pero utilizando la función definida.

Este enfoque es muy útil para calcular la frecuencia de diferentes combinaciones de productos en las transacciones, un paso esencial en el análisis de reglas de asociación.

In [57]:
def conteo(X, T):
    X_subset =  T["Transaction"].apply(lambda t: X.issubset(t))
    return sum(X_subset)

In [58]:
conteo(set(["LOVE BUILDING BLOCK WORD"]), T)

557

## **Reglas de asociación**

Las **reglas de asociación** son herramientas utilizadas en el análisis de datos para descubrir relaciones interesantes entre variables en grandes bases de datos. Son especialmente populares en el análisis de transacciones de comercio, como el análisis de cestas de compra en supermercados, donde el objetivo es encontrar patrones en las compras de los clientes.

### ¿Cómo Funcionan?

Una regla de asociación se compone de dos partes: un antecedente (A) y un consecuente (C), que se encuentran en la forma A -> C. La interpretación es que cuando ocurre A, es probable que ocurra C. Por ejemplo, si A representa la compra de pan y C representa la compra de mantequilla, la regla sugiere que si un cliente compra pan, es probable que también compre mantequilla.

### Métricas Clave

Las reglas de asociación se evalúan utilizando tres métricas clave:

- **Soporte**: Indica la frecuencia con la que aparece un ítem o una combinación de ítems en el conjunto de datos. Por ejemplo, el soporte de A -> C es la proporción de transacciones en los datos que contienen tanto A como C.

- **Confianza**: Mide la frecuencia con la que los ítems en C aparecen en transacciones que contienen A. Es decir, cuántas veces la presencia de A implica la presencia de C.

- **Elevación (Lift)**: Mide cuánto más a menudo A y C ocurren juntos de lo que se esperaría si fueran estadísticamente independientes. Una elevación mayor a 1 indica una relación positiva entre A y C.


### Soporte de una Regla

El **soporte de una regla** es una métrica utilizada en el análisis de reglas de asociación para determinar la frecuencia relativa de aparición de un conjunto de ítems en un conjunto de datos de transacciones. Se calcula como la proporción de transacciones que contienen todos los ítems en la regla (tanto en el antecedente como en el consecuente) respecto al total de transacciones en el conjunto de datos.

#### Fórmula

El soporte de una regla A -> C se calcula con la siguiente fórmula:

$$
\text{Soporte}(A \rightarrow C) = \frac{\text{Número de transacciones que contienen } A \cup C}{\text{Número total de transacciones}}
$$

Donde:
- \(A\) es el antecedente de la regla.
- \(C\) es el consecuente de la regla.
- \(A \cup C\) representa la unión de los ítems en A y C.

#### Importancia del Soporte

- **Popularidad**: Un soporte alto indica que la combinación de ítems es más común en el conjunto de datos.
- **Filtrado**: Ayuda a eliminar reglas que involucran ítems raramente comprados juntos, enfocándose en relaciones más significativas o comunes.
- **Bases para otras métricas**: El soporte es fundamental para calcular otras métricas importantes como la confianza y la elevación.

#### Ejemplo

Si tenemos 1000 transacciones en total y la combinación de pan y mantequilla (A -> C) aparece en 100 de estas transacciones, entonces el soporte de la regla "si se compra pan entonces se compra mantequilla" es 100/1000 o 0.1 (10%).

Este valor de soporte nos indica que el 10% de todas las transacciones en nuestro conjunto de datos incluyen tanto el pan como la mantequilla.

In [59]:
def soporte(X,Y,T):
    XuY = X.union(Y)
    return conteo(XuY, T)/len(T)

1. **Definición de la función `soporte`:**
   - `X` y `Y` son conjuntos que representan el antecedente y el consecuente de la regla de asociación, respectivamente.
   - `T` es el dataframe que contiene todas las transacciones, cada una representada como un conjunto de ítems.

In [60]:
X = set(["LOVE BUILDING BLOCK WORD"])
Y = set(["RECIPE BOX WITH METAL HEART"])

soporte(X,Y,T)

4.813941173638858e-05

El resultado `4.813941173638858e-05` indica la proporción de transacciones en el conjunto de datos \(T\) que contienen tanto "LOVE BUILDING BLOCK WORD" como "RECIPE BOX WITH METAL HEART". En este caso, el soporte es extremadamente bajo, lo que significa que la ocurrencia conjunta de estos dos productos en el mismo conjunto de transacciones es muy rara dentro del conjunto de datos.

- Un soporte de \(0.00004813941173638858\) sugiere que de cada 10,000 transacciones, aproximadamente \(0.4813941173638858\) (o prácticamente 0) contienen ambos ítems. Esto se traduce en que esta combinación de productos es bastante inusual.
- En términos de análisis de reglas de asociación, una regla con un soporte tan bajo podría no ser considerada significativa o útil para tomar decisiones estratégicas, a menos que el contexto específico de los datos justifique lo contrario.

## **Confianza**

La **confianza** es una métrica crucial en el análisis de reglas de asociación que mide la fuerza de la implicación de una regla. Se define como la probabilidad de ver el conjunto de ítems Y en transacciones dado que estas transacciones ya contienen X. En términos más técnicos, la confianza se calcula como la proporción de transacciones con X que también contienen Y.

### Fórmula de la Confianza

La fórmula para calcular la confianza de una regla \(X --> Y\) es:

$$
\text{confidence}_{X\rightarrow Y} = \frac{\text{Soporte}(X \cup Y)}{\text{Soporte}(X)}
$$

o de manera equivalente:

$$
\text{confidence}_{X\rightarrow Y} = \frac{\text{Número de transacciones que contienen ambos, } X \text{ y } Y}{\text{Número de transacciones que contienen } X}
$$

### Interpretación

- Una **confianza alta** significa que es más probable que Y esté presente en transacciones que ya contienen X.
- Por ejemplo, una confianza del 80% en la regla \(X \rightarrow Y\) significa que en el 80% de las transacciones que contienen X, también se encuentra Y.
- Es importante tener en cuenta que la confianza puede ser engañosa si el ítem Y es muy común en general, ya que esto podría inflar la confianza sin indicar una asociación real entre X y Y.

### Importancia

- La confianza ayuda a identificar reglas que son potencialmente útiles para predecir la presencia de ciertos ítems en conjunto con otros.
- Es una medida de la fiabilidad de la regla; una confianza más alta indica una regla más fuerte.


La función `confianza` que has definido en Python está diseñada para calcular la confianza de una regla de asociación específica en un conjunto de datos representado por transacciones. La confianza es una medida de cuán frecuentemente los ítems en el consecuente (Y) ocurren en transacciones que también contienen el antecedente (X). Aquí está el detalle de lo que hace cada parte de la función y cómo se aplica al ejemplo que proporcionaste:

### Definición de la Función `confianza`

- **Entradas de la función:**
  - `X`: Un conjunto de elementos que forman el antecedente de la regla.
  - `Y`: Un conjunto de elementos que forman el consecuente de la regla.
  - `T`: El conjunto de datos de transacciones, donde cada transacción es un conjunto de elementos.

- **Proceso de la función:**
  - `XuY = X.union(Y)`: Crea un nuevo conjunto `XuY` que es la unión de `X` y `Y`, representando todos los elementos presentes en el antecedente y en el consecuente.
  - `return conteo(XuY, T)/conteo(X, T)`: Calcula la confianza como la proporción entre el número de transacciones que contienen tanto `X` como `Y` (usando `conteo(XuY, T)`) y el número de transacciones que contienen `X` (usando `conteo(X, T)`).

In [61]:
def confianza(X,Y,T):
    XuY = X.union(Y)
    return conteo(XuY, T)/conteo(X, T)

### Ejemplo de Aplicación

- **Regla a evaluar:**
  - Se quiere evaluar la confianza de la regla que dice: si un cliente compra "LOVE BUILDING BLOCK WORD", entonces también comprará "RECIPE BOX WITH METAL HEART".

- **Cálculo de la confianza:**
  - `confianza(set(["LOVE BUILDING BLOCK WORD"]), set(["RECIPE BOX WITH METAL HEART"]), T)`: Esta llamada a la función calcula la confianza de la regla mencionada utilizando el conjunto de datos de transacciones `T`.

### Interpretación del Resultado

- El resultado `0.0017953321364452424` indica la confianza de la regla. En este caso, es un valor bastante bajo, lo que significa que, aunque algunos clientes que compran "LOVE BUILDING BLOCK WORD" también compran "RECIPE BOX WITH METAL HEART", esta tendencia no es muy fuerte en el conjunto de datos.
- En términos porcentuales, la confianza de aproximadamente 0.18% sugiere que de cada 1000 clientes que compran "LOVE BUILDING BLOCK WORD", solo alrededor de 1.8 también comprarían "RECIPE BOX WITH METAL HEART".

Este bajo nivel de confianza sugiere que la implicación de comprar "RECIPE BOX WITH METAL HEART" después de "LOVE BUILDING BLOCK WORD" no es una regla fuerte en este conjunto de datos, indicando que estas dos acciones de compra no están tan fuertemente asociadas.

In [62]:
confianza(set(["LOVE BUILDING BLOCK WORD"]), set(["RECIPE BOX WITH METAL HEART"]), T)

0.0017953321364452424

## **Objetivo**

Identificar todas las reglas de asociación cuyo soporte sea al menos igual a un umbral mínimo predefinido (\(\text{supp}_{min}\)). Este umbral ayuda a filtrar reglas que son relevantes y descartar aquellas que son demasiado infrecuentes para ser consideradas significativas.

### Conjuntos Frecuentes

- **Conjuntos frecuentes** son aquellos conjuntos de ítems (o artículos) que aparecen juntos en una proporción de transacciones mayor o igual al soporte mínimo (\(\text{supp}_{min}\)).
- La identificación de estos conjuntos es un paso previo esencial para luego poder formular y evaluar las reglas de asociación.

### Estrategia de Fuerza Bruta

- La estrategia de **fuerza bruta** implica examinar todas las posibles combinaciones de ítems para determinar cuáles cumplen con el criterio de soporte mínimo.
- Es un enfoque exhaustivo que, aunque garantiza que no se pasará por alto ningún conjunto frecuente, puede ser muy ineficiente en términos de tiempo de cálculo, especialmente con grandes conjuntos de datos.

### Ejemplo con Soporte Mínimo de 0.3

- Establecemos un soporte mínimo de \(0.3\), lo que significa que solo consideraremos como conjuntos frecuentes aquellos que aparezcan en al menos el 30% de todas las transacciones.
- Comenzamos con conjuntos de un solo elemento, evaluando la frecuencia de cada ítem individualmente en el conjunto de datos para determinar si su soporte alcanza o supera el 0.3.

In [63]:
Items = list(I)

Convierte el conjunto I, que contiene todos los ítems únicos en el conjunto de datos, en una lista llamada Items. Esto se hace porque la función combinations necesita una lista (o un objeto similar) para trabajar.

In [64]:
sets_1 = combinations(Items,1)

for s in sets_1:
    X = set([s[0]])
    print(X)

{' 4 PURPLE FLOCK DINNER CANDLES'}
{" 50'S CHRISTMAS GIFT BAG LARGE"}
{' DOLLY GIRL BEAKER'}
{' I LOVE LONDON MINI BACKPACK'}
{' I LOVE LONDON MINI RUCKSACK'}
{' NINE DRAWER OFFICE TIDY'}
{' OVAL WALL MIRROR DIAMANTE '}
{' RED SPOT GIFT BAG LARGE'}
{' SET 2 TEA TOWELS I LOVE LONDON '}
{' SPACEBOY BABY GIFT SET'}
{' TOADSTOOL BEDSIDE LIGHT '}
{' TRELLIS COAT RACK'}
{'10 COLOUR SPACEBOY PEN'}
{'12 COLOURED PARTY BALLOONS'}
{'12 DAISY PEGS IN WOOD BOX'}
{'12 EGG HOUSE PAINTED WOOD'}
{'12 HANGING EGGS HAND PAINTED'}
{'12 IVORY ROSE PEG PLACE SETTINGS'}
{'12 MESSAGE CARDS WITH ENVELOPES'}
{'12 PENCIL SMALL TUBE WOODLAND'}
{'12 PENCILS SMALL TUBE RED RETROSPOT'}
{'12 PENCILS SMALL TUBE SKULL'}
{'12 PENCILS TALL TUBE POSY'}
{'12 PENCILS TALL TUBE RED RETROSPOT'}
{'12 PENCILS TALL TUBE SKULLS'}
{'12 PENCILS TALL TUBE WOODLAND'}
{'12 PINK HEN+CHICKS IN BASKET'}
{'12 PINK ROSE PEG PLACE SETTINGS'}
{'12 RED ROSE PEG PLACE SETTINGS'}
{'15 PINK FLUFFY CHICKS IN BOX'}
{'15CM CHRISTMAS GLASS BALL 20 

1. **Generación de subconjuntos de un elemento:**  
   `sets = combinations(Items, 1)`  
   Esta línea utiliza la función `combinations` para generar todas las posibles combinaciones de un elemento de la lista `Items`. Cada combinación es una tupla de un elemento que representa un posible conjunto de ítems.

2. **Establecimiento del soporte mínimo:**  
   `supp_min = 0.05`  
   Define el soporte mínimo como 0.05, lo que significa que solo se considerarán los conjuntos de ítems que aparecen en al menos el 5% de todas las transacciones.

3. **Iteración y evaluación del soporte:**  
   El bucle `for s in sets:` recorre cada tupla generada por `combinations`.  
   Dentro del bucle:
   - `X = set([s[0]])` convierte la tupla en un conjunto para poder calcular su soporte.
   - `supp = soporte(X, X, T)` calcula el soporte del conjunto `X`. Aquí, `soporte(X, X, T)` es equivalente a `soporte(X, T)` ya que estás evaluando el soporte de un solo elemento.
   - Si `supp >= supp_min`, se verifica si el soporte de `X` es igual o superior al soporte mínimo. Si lo es, el conjunto `X` se imprime.

4. **Impresión de conjuntos frecuentes:**  
   Si el soporte de `X` es mayor o igual al soporte mínimo, `X` se imprime, indicando que es un conjunto frecuente según el umbral definido.

In [65]:
sets = combinations(Items,1)

supp_min = 0.05

for s in sets:
    X = set([s[0]])
    supp = soporte(X,X,T)
    if supp >= supp_min:
        print(X)

{'ASSORTED COLOUR BIRD ORNAMENT'}
{'JUMBO BAG RED RETROSPOT'}
{'LUNCH BAG  BLACK SKULL.'}
{'LUNCH BAG RED RETROSPOT'}
{'PARTY BUNTING'}
{'POSTAGE'}
{'REGENCY CAKESTAND 3 TIER'}
{'SET OF 3 CAKE TINS PANTRY DESIGN '}
{'WHITE HANGING HEART T-LIGHT HOLDER'}


## **Algoritmo Apriori**

El algoritmo Apriori es una técnica clásica en minería de datos para la extracción de conjuntos frecuentes y la generación de reglas de asociación. Es ampliamente utilizado para el análisis de cestas de compra, donde ayuda a descubrir qué productos tienden a comprarse juntos. Aquí te explico cómo funciona el algoritmo:

### Principios Básicos del Algoritmo Apriori

1. **Conjuntos Frecuentes:** Un conjunto frecuente es un conjunto de ítems que aparecen juntos en un número suficiente de transacciones, superando un umbral de soporte mínimo establecido.

2. **Propiedad Apriori:** Esta propiedad fundamental del algoritmo establece que si un conjunto de ítems es frecuente, entonces todos sus subconjuntos no vacíos también deben ser frecuentes. Inversamente, si un conjunto de ítems no es frecuente, entonces todos sus superconjuntos serán infrecuentes.

### Pasos del Algoritmo Apriori

1. **Inicialización:** Comienza con la identificación de todos los conjuntos de un elemento que cumplen con el soporte mínimo.

2. **Generación de Conjuntos Candidatos:** En cada iteración \(k\), el algoritmo genera nuevos conjuntos candidatos de tamaño \(k\) combinando conjuntos frecuentes de tamaño \(k-1\) que comparten \(k-2\) elementos en común.

3. **Cálculo del Soporte:** Calcula el soporte de cada conjunto candidato contando las apariciones en el conjunto de datos y compara este soporte con el umbral mínimo. Los conjuntos que no cumplen con el soporte mínimo se descartan.

4. **Iteración:** El proceso se repite incrementando el tamaño de los conjuntos en cada iteración hasta que no se puedan formar nuevos conjuntos frecuentes.

5. **Generación de Reglas de Asociación:** Una vez identificados todos los conjuntos frecuentes, el algoritmo utiliza estos conjuntos para generar reglas de asociación. Para cada conjunto frecuente, se generan todas las posibles reglas de partición que cumplen con un umbral mínimo de confianza.

6. **Evaluación de Reglas:** Las reglas generadas se evalúan basadas en la confianza y otros parámetros como el lift, y solo se conservan las reglas que cumplen con los criterios establecidos.

### Ejemplo de Aplicación

Imagina que tienes un conjunto de datos de transacciones de una tienda. Al aplicar Apriori, puedes descubrir que los clientes que compran pan y mantequilla a menudo también compran leche (pan, mantequilla) → leche. Si este conjunto de ítems aparece en más transacciones de las que dicta el soporte mínimo y la regla cumple con la confianza mínima, se considerará una regla de asociación significativa.

El algoritmo Apriori es potente para encontrar relaciones ocultas en grandes conjuntos de datos, pero su eficiencia puede verse afectada cuando se trabaja con conjuntos de datos muy grandes y con un número elevado de transacciones, debido a la gran cantidad de conjuntos candidatos que puede generar.

### Conjuntos frecuentes (un solo candidato)

El siguiente código está diseñado para encontrar y almacenar todos los conjuntos frecuentes de un solo elemento basándose en un umbral de soporte mínimo definido. En el contexto del algoritmo Apriori, estos serían los primeros conjuntos frecuentes identificados (denominados F1) que se usarán para construir conjuntos frecuentes más grandes en las iteraciones subsiguientes.

In [66]:
sets = combinations(Items,1)

supp_min = 0.04

F1 = []

for s in sets:
    X = set([s[0]])
    supp = soporte(X,X,T)
    if supp >= supp_min:
        F1.append(X)
F1

[{'ALARM CLOCK BAKELIKE RED '},
 {'ASSORTED COLOUR BIRD ORNAMENT'},
 {'BAKING SET 9 PIECE RETROSPOT '},
 {'HEART OF WICKER SMALL'},
 {'JAM MAKING SET PRINTED'},
 {'JAM MAKING SET WITH JARS'},
 {'JUMBO BAG PINK POLKADOT'},
 {'JUMBO BAG RED RETROSPOT'},
 {'LUNCH BAG  BLACK SKULL.'},
 {'LUNCH BAG APPLE DESIGN'},
 {'LUNCH BAG CARS BLUE'},
 {'LUNCH BAG PINK POLKADOT'},
 {'LUNCH BAG RED RETROSPOT'},
 {'LUNCH BAG SPACEBOY DESIGN '},
 {'LUNCH BAG SUKI DESIGN '},
 {'LUNCH BAG WOODLAND'},
 {'NATURAL SLATE HEART CHALKBOARD '},
 {'PACK OF 72 RETROSPOT CAKE CASES'},
 {"PAPER CHAIN KIT 50'S CHRISTMAS "},
 {'PARTY BUNTING'},
 {'POSTAGE'},
 {'RECIPE BOX PANTRY YELLOW DESIGN'},
 {'REGENCY CAKESTAND 3 TIER'},
 {'RETROSPOT TEA SET CERAMIC 11 PC '},
 {'REX CASH+CARRY JUMBO SHOPPER'},
 {'ROSES REGENCY TEACUP AND SAUCER '},
 {'SET OF 3 CAKE TINS PANTRY DESIGN '},
 {'SET OF 4 PANTRY JELLY MOULDS'},
 {'SPOTTY BUNTING'},
 {'VICTORIAN GLASS HANGING T-LIGHT'},
 {'WHITE HANGING HEART T-LIGHT HOLDER'},
 {'WOODEN P

El resultado de tu código muestra una lista `F1` que contiene los conjuntos frecuentes de un solo elemento cuyo soporte es mayor o igual al soporte mínimo establecido en 0.04. Cada elemento en la lista `F1` es un conjunto (`set`) que incluye un único producto. Estos productos son aquellos que aparecen en al menos el 4% de todas las transacciones en el conjunto de datos `T`.

Estos conjuntos frecuentes representan los ítems individuales que son significativamente populares en el conjunto de datos. Son la base para la siguiente etapa del algoritmo Apriori, donde se combinarán estos ítems frecuentes para encontrar conjuntos frecuentes de mayor tamaño (por ejemplo, conjuntos de dos elementos, luego de tres, y así sucesivamente), siempre aplicando la propiedad Apriori para reducir el espacio de búsqueda y mejorar la eficiencia del algoritmo.

Identificar estos conjuntos frecuentes ayuda a entender cuáles son los productos más populares y sentar las bases para descubrir reglas de asociación que puedan indicar cómo la compra de ciertos productos influye en la compra de otros.

### Conjuntos frecuentes (dos cadidatos)

El fragmento de código está diseñado para generar conjuntos de dos elementos, denominados C2 (candidatos de tamaño 2), a partir de los conjuntos frecuentes de un solo elemento (F1) que ya identificaste.

In [67]:
C2 = []

for i,s in enumerate(F1):
    for j,t in enumerate(F1):
        if j > i:
            sUt = s.union(t)
            print(sorted(sUt))
            C2.append(set(sUt))

['ALARM CLOCK BAKELIKE RED ', 'ASSORTED COLOUR BIRD ORNAMENT']
['ALARM CLOCK BAKELIKE RED ', 'BAKING SET 9 PIECE RETROSPOT ']
['ALARM CLOCK BAKELIKE RED ', 'HEART OF WICKER SMALL']
['ALARM CLOCK BAKELIKE RED ', 'JAM MAKING SET PRINTED']
['ALARM CLOCK BAKELIKE RED ', 'JAM MAKING SET WITH JARS']
['ALARM CLOCK BAKELIKE RED ', 'JUMBO BAG PINK POLKADOT']
['ALARM CLOCK BAKELIKE RED ', 'JUMBO BAG RED RETROSPOT']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG  BLACK SKULL.']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG APPLE DESIGN']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG CARS BLUE']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG PINK POLKADOT']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG RED RETROSPOT']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG SPACEBOY DESIGN ']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG SUKI DESIGN ']
['ALARM CLOCK BAKELIKE RED ', 'LUNCH BAG WOODLAND']
['ALARM CLOCK BAKELIKE RED ', 'NATURAL SLATE HEART CHALKBOARD ']
['ALARM CLOCK BAKELIKE RED ', 'PACK OF 72 RETROSPOT CAKE CASES']
['ALARM CLOCK

Se han generado 496 pares únicos de elementos al combinar los conjuntos frecuentes de un solo elemento. Estos pares representan los conjuntos candidatos de dos elementos que fueron considerados para evaluar si su soporte es suficientemente alto para clasificarlos como conjuntos frecuentes.

In [68]:
len(C2)

496

In [69]:
F2 = []

for s in C2:
    supp = soporte(s,s,T)
    if supp >= supp_min:
        F2.append(s)
F2

[]

No hay conjuntos de dos elementos que superen el umbral de soporte mínimo establecido. Esto significa que, aunque individualmente algunos elementos pueden ser frecuentes, no hay pares de elementos que aparezcan juntos en las transacciones lo suficientemente a menudo como para considerarse frecuentes bajo el criterio de soporte mínimo actual. Esto puede deberse a un umbral de soporte demasiado alto o a la naturaleza dispersa de los datos.

In [70]:
len(F2)

0

## **Reglas de asociación**

En este proceso generamos reglas de asociación utilizando el algoritmo Apriori en un conjunto de datos de transacciones. Aquí se emplea la librería `mlxtend` en Python, que ofrece una implementación clara y eficiente del algoritmo.

### 1. Preparación de los Datos
Utilizamos `TransactionEncoder` para transformar los datos de transacciones en un formato adecuado para el análisis de reglas de asociación. `TransactionEncoder` convierte los datos en un dataframe donde cada fila representa una transacción y cada columna representa un ítem, con valores booleanos indicando la presencia o ausencia de cada ítem en cada transacción.

### 2. Aplicación del Algoritmo Apriori
Luego, aplicamos la función `apriori` para identificar los conjuntos frecuentes de ítems en el dataframe transformado. Estableces un soporte mínimo de 0.02 para filtrar los conjuntos frecuentes, adaptando el soporte mínimo para obtener una cantidad razonable de conjuntos frecuentes para el análisis.

### 3. Generación de Reglas de Asociación
Con los conjuntos frecuentes encontrados, utilizas la función `association_rules` para generar reglas de asociación. Especificas la métrica de "confidence" (confianza) y un umbral mínimo de 0.7 para filtrar las reglas según su confianza. La confianza es una medida de cuán frecuentemente los ítems en el consecuente de la regla ocurren en las transacciones que contienen el antecedente.

In [73]:
te = TransactionEncoder()
te_ary = te.fit(T['Transaction']).transform(T['Transaction'])
df = pd.DataFrame(te_ary, columns=te.columns_)

In [79]:
# nd = df.to_csv('../files/Transaction.csv', index=False)

In [75]:
df.head(10)


,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [76]:
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
frequent_itemsets.head(10)

,support,itemsets
0,0.035190,(6 RIBBONS RUSTIC CHARM)
1,0.022481,(60 CAKE CASES VINTAGE CHRISTMAS)
2,0.031965,(60 TEATIME FAIRY CAKE CASES)
3,0.024310,(72 SWEETHEART FAIRY CAKE CASES)
4,0.038704,(ALARM CLOCK BAKELIKE GREEN)
5,0.023011,(ALARM CLOCK BAKELIKE IVORY)
6,0.030183,(ALARM CLOCK BAKELIKE PINK)
7,0.043422,(ALARM CLOCK BAKELIKE RED )
8,0.026766,(ANTIQUE SILVER T-LIGHT GLASS)
9,0.066625,(ASSORTED COLOUR BIRD ORNAMENT)


In [77]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(GARDENERS KNEELING PAD CUP OF TEA ),(GARDENERS KNEELING PAD KEEP CALM ),0.030713,0.036779,0.022337,0.727273,19.774393,0.021207,3.531812,0.979513
1,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.028162,0.035094,0.022577,0.801709,22.844869,0.021589,4.866123,0.983936
2,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.035094,0.040100,0.026717,0.761317,18.985397,0.025310,4.021649,0.981782
3,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.028162,0.040100,0.021615,0.767521,19.140122,0.020485,4.128981,0.975217


In [78]:
result = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
# result.to_csv('../files/association_rules.csv', index=False)

### Resultados
Los resultados muestran varias reglas de asociación junto con métricas clave:
- **Antecedentes** y **consecuentes**: Los ítems en el lado izquierdo y derecho de la regla, respectivamente.
- **Soporte del antecedente y del consecuente**: La proporción de transacciones que contienen el antecedente y el consecuente, respectivamente.
- **Soporte**: La proporción de transacciones que contienen tanto el antecedente como el consecuente.
- **Confianza**: La proporción de transacciones con el antecedente que también contienen el consecuente.
- **Lift**: Una medida de cuánto más a menudo el antecedente y el consecuente ocurren juntos de lo que se esperaría si fueran estadísticamente independientes.
- **Leverage** y **Conviction**: Otras métricas que ofrecen perspectivas adicionales sobre la fuerza y la relevancia de la regla.

## **Análisis de las reglas de asociación**

Analizaremos cada elemento de las reglas de asociación proporcionadas, utilizando las reglas listadas como ejemplos:

### 1. Primera Regla:
- **Antecedente (antecedents):** `{GARDENERS KNEELING PAD CUP OF TEA }`
- **Consecuente (consequents):** `{GARDENERS KNEELING PAD KEEP CALM }`
- **Soporte del Antecedente (antecedent support):** 0.0307 (3.07%)
- **Soporte del Consecuente (consequent support):** 0.0368 (3.68%)
- **Soporte (support):** 0.0223 (2.23%)
- **Confianza (confidence):** 0.7273 (72.73%)
- **Lift (lift):** 19.7744
- **Leverage (leverage):** 0.0212
- **Conviction (conviction):** 3.5318
- **Zhang's Metric (zhangs_metric):** 0.9795

Esta regla indica una fuerte relación entre dos productos relacionados con jardinería. La alta confianza y el lift sugieren que cuando los clientes compran un tipo de almohadilla para arrodillarse, es muy probable que también compren el otro tipo. La métrica de Zhang confirma esta fuerte asociación.

### 2. Segunda Regla:
- **Antecedente:** `{PINK REGENCY TEACUP AND SAUCER}`
- **Consecuente:** `{GREEN REGENCY TEACUP AND SAUCER}`
- **Soporte del Antecedente:** 0.0282 (2.82%)
- **Soporte del Consecuente:** 0.0351 (3.51%)
- **Soporte:** 0.0226 (2.26%)
- **Confianza:** 0.8017 (80.17%)
- **Lift:** 22.8449
- **Leverage:** 0.0216
- **Conviction:** 4.8661
- **Zhang's Metric:** 0.9839

Esta regla muestra una relación fuerte entre dos tazas de té de colores diferentes. La confianza muy alta indica que la mayoría de los clientes que compran la taza rosa también compran la verde.

### 3. Tercera Regla:
- **Antecedente:** `{GREEN REGENCY TEACUP AND SAUCER}`
- **Consecuente:** `{ROSES REGENCY TEACUP AND SAUCER }`
- **Soporte del Antecedente:** 0.0351 (3.51%)
- **Soporte del Consecuente:** 0.0401 (4.01%)
- **Soporte:** 0.0267 (2.67%)
- **Confianza:** 0.7613 (76.13%)
- **Lift:** 18.9854
- **Leverage:** 0.0253
- **Conviction:** 4.0216
- **Zhang's Metric:** 0.9818

Esta regla indica que los clientes que compran la taza verde tienden a comprar también la taza con rosas. La relación es significativa, como lo demuestra el lift y la confianza.

### 4. Cuarta Regla:
- **Antecedente:** `{PINK REGENCY TEACUP AND SAUCER}`
- **Consecuente:** `{ROSES REGENCY TEACUP AND SAUCER }`
- **Soporte del Antecedente:** 0.0282 (2.82%)
- **Soporte del Consecuente:** 0.0401 (4.01%)
- **Soporte:** 0.0216 (2.16%)
- **Confianza:** 0.7675 (76.75%)
- **Lift:** 19.1401
- **Leverage:** 0.0205
- **Conviction:** 4.1290
- **Zhang's Metric:** 0.9752

Similar a la segunda regla, esta muestra que la compra de la taza de té rosa está fuertemente asociada con la compra de la taza con diseño de rosas.

### Conclusión General

Todas estas reglas tienen un lift significativamente mayor que 1, lo que indica que las combinaciones de productos son más comunes de lo que serían si fueran independientes. Las altas confianzas sugieren que la presencia del antecedente aumenta la probabilidad de que también se compre el consecuente. Es decir, cuando los clientes compran el producto o conjunto de productos en el antecedente, es mucho más probable que también adquieran el producto en el consecuente, en comparación con la probabilidad de compra del consecuente en general.

Además, valores de leverage positivos indican que las combinaciones de ítems en estas reglas ocurren más frecuentemente juntas de lo esperado; esto refuerza la idea de una relación positiva entre los ítems. La métrica de conviction ofrece una mirada adicional sobre la fortaleza de esta implicación: un valor alto sugiere que es muy probable que la presencia del antecedente influya en la aparición del consecuente en las transacciones.

La métrica de Zhang proporciona una confirmación adicional de la fuerza y la dirección de la relación. Valores cercanos a 1 implican una asociación positiva fuerte, lo que significa que la presencia del antecedente tiene una influencia significativa en la presencia del consecuente.

Estas métricas combinadas presentan una imagen clara: hay una relación significativa y positiva entre los productos en el antecedente y el consecuente, lo que puede ser utilizado estratégicamente. Por ejemplo, en el marketing y la colocación de productos, los minoristas podrían colocar los productos del antecedente y del consecuente cerca uno del otro para aumentar las ventas cruzadas. En la gestión de inventario, comprender estas relaciones puede ayudar a optimizar las existencias basándose en la demanda anticipada de productos asociados.